# Deep Q-Network (DQN) 深度解析

本 Notebook 深入剖析 DQN 算法的核心组件和数学原理，通过交互式实验帮助理解算法细节。

## 目录
1. [DQN 算法概述](#1-dqn-算法概述)
2. [Q 函数与贝尔曼方程](#2-q-函数与贝尔曼方程)
3. [经验回放机制](#3-经验回放机制)
4. [目标网络](#4-目标网络)
5. [ε-贪婪探索](#5-ε-贪婪探索)
6. [DQN 变体对比](#6-dqn-变体对比)
7. [实验与分析](#7-实验与分析)

In [ ]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from typing import Tuple

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# TF-Agents
from tf_agents.environments import suite_gym
from tf_agents.environments import tf_py_environment
from tf_agents.networks import q_network
from tf_agents.agents.dqn import dqn_agent
from tf_agents.replay_buffers import tf_uniform_replay_buffer
from tf_agents.trajectories import trajectory
from tf_agents.policies import random_tf_policy
from tf_agents.utils import common

np.random.seed(42)
tf.random.set_seed(42)

## 1. DQN 算法概述

### 1.1 核心思想

DQN (Deep Q-Network) 使用深度神经网络逼近 Q 函数，解决了传统 Q-Learning 无法处理高维状态空间的问题。

**关键创新**:
1. **经验回放 (Experience Replay)**: 打破样本相关性
2. **目标网络 (Target Network)**: 稳定训练目标

### 1.2 算法流程

```
初始化 Q 网络 θ 和目标网络 θ⁻
初始化经验回放缓冲区 D

for episode = 1 to M:
    初始化状态 s₁
    for t = 1 to T:
        # ε-贪婪动作选择
        以概率 ε 选择随机动作 aₜ
        否则选择 aₜ = argmax_a Q(sₜ, a; θ)
        
        执行动作 aₜ，观测奖励 rₜ 和下一状态 sₜ₊₁
        存储 (sₜ, aₜ, rₜ, sₜ₊₁) 到 D
        
        # 从 D 采样 mini-batch
        计算目标: yⱼ = rⱼ + γ max_a' Q(s'ⱼ, a'; θ⁻)
        最小化损失: L = (yⱼ - Q(sⱼ, aⱼ; θ))²
        
        # 周期性更新目标网络
        每 C 步: θ⁻ ← θ
```

## 2. Q 函数与贝尔曼方程

### 2.1 Q 函数定义

状态-动作价值函数 $Q^\pi(s, a)$ 表示从状态 $s$ 执行动作 $a$，然后遵循策略 $\pi$ 的期望累积奖励：

$$Q^\pi(s, a) = \mathbb{E}_\pi\left[\sum_{t=0}^{\infty} \gamma^t r_t | s_0 = s, a_0 = a\right]$$

### 2.2 最优贝尔曼方程

最优 Q 函数满足贝尔曼最优方程：

$$Q^*(s, a) = \mathbb{E}_{s'}\left[r + \gamma \max_{a'} Q^*(s', a') | s, a\right]$$

这是一个递归定义：当前状态-动作的价值 = 即时奖励 + 折扣后的最优未来价值。

In [ ]:
# 演示贝尔曼方程的迭代求解（简单网格世界）

def bellman_iteration_demo():
    """
    在简单的 3x3 网格世界中演示贝尔曼迭代
    
    网格布局:
    [0][1][2]
    [3][4][5]
    [6][7][G]  G=目标状态(+10奖励)
    """
    num_states = 9
    num_actions = 4  # 上下左右
    gamma = 0.9
    
    # Q 值表初始化
    Q = np.zeros((num_states, num_actions))
    
    # 奖励：到达目标状态(8)获得+10
    rewards = np.zeros(num_states)
    rewards[8] = 10.0
    
    # 简单的转移函数（确定性）
    # transitions[s][a] = next_state
    transitions = np.array([
        [0, 3, 0, 1],  # 状态0: 上→0, 下→3, 左→0, 右→1
        [1, 4, 0, 2],
        [2, 5, 1, 2],
        [0, 6, 3, 4],
        [1, 7, 3, 5],
        [2, 8, 4, 5],
        [3, 6, 6, 7],
        [4, 8, 6, 8],
        [8, 8, 8, 8],  # 目标状态（终止）
    ])
    
    # 贝尔曼迭代
    Q_history = [Q.copy()]
    
    for iteration in range(20):
        Q_new = np.zeros_like(Q)
        
        for s in range(num_states):
            for a in range(num_actions):
                s_next = transitions[s, a]
                # Q(s,a) = r + γ * max_a' Q(s', a')
                Q_new[s, a] = rewards[s_next] + gamma * np.max(Q[s_next])
        
        Q = Q_new
        Q_history.append(Q.copy())
    
    return Q, Q_history

Q_final, Q_history = bellman_iteration_demo()

# 可视化收敛过程
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 左图：Q 值收敛过程
ax1 = axes[0]
for s in [4, 5, 7]:  # 选择几个状态可视化
    max_q_over_time = [np.max(Q_history[i][s]) for i in range(len(Q_history))]
    ax1.plot(max_q_over_time, label=f'State {s}')

ax1.set_xlabel('Iteration')
ax1.set_ylabel('max Q(s, a)')
ax1.set_title('Bellman Iteration Convergence')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 右图：最终 Q 值热力图
ax2 = axes[1]
V = np.max(Q_final, axis=1).reshape(3, 3)
im = ax2.imshow(V, cmap='YlOrRd')
ax2.set_title('State Values V(s) = max_a Q(s,a)')
for i in range(3):
    for j in range(3):
        ax2.text(j, i, f'{V[i,j]:.1f}', ha='center', va='center')
plt.colorbar(im, ax=ax2)

plt.tight_layout()
plt.show()

print("最终 Q 值表:")
print("动作: [上, 下, 左, 右]")
print(Q_final)

## 3. 经验回放机制

### 3.1 为什么需要经验回放？

直接使用在线数据训练神经网络存在两个问题：

1. **样本相关性**: 连续的状态高度相关，违反 i.i.d. 假设
2. **数据效率**: 每个经验只使用一次，浪费了有价值的数据

### 3.2 经验回放的作用

- 存储历史经验 $(s, a, r, s')$
- 随机采样打破时间相关性
- 允许多次重用数据

In [ ]:
# 演示经验回放的采样分布

class SimpleReplayBuffer:
    """简单的经验回放缓冲区实现"""
    
    def __init__(self, capacity):
        self.capacity = capacity
        self.buffer = []
        self.position = 0
    
    def add(self, experience):
        if len(self.buffer) < self.capacity:
            self.buffer.append(experience)
        else:
            self.buffer[self.position] = experience
        self.position = (self.position + 1) % self.capacity
    
    def sample(self, batch_size):
        indices = np.random.choice(len(self.buffer), batch_size, replace=False)
        return [self.buffer[i] for i in indices], indices

# 模拟数据收集过程
buffer = SimpleReplayBuffer(capacity=1000)

# 添加模拟经验（带时间戳）
for t in range(1000):
    experience = {'time': t, 'data': np.random.randn()}
    buffer.add(experience)

# 采样并分析时间分布
sampled_times = []
for _ in range(100):  # 100 次采样
    batch, indices = buffer.sample(batch_size=64)
    times = [exp['time'] for exp in batch]
    sampled_times.extend(times)

# 可视化
plt.figure(figsize=(10, 4))
plt.hist(sampled_times, bins=50, edgecolor='black', alpha=0.7)
plt.xlabel('Experience Timestamp')
plt.ylabel('Sample Count')
plt.title('Uniform Sampling from Replay Buffer\n(Breaks temporal correlation)')
plt.grid(True, alpha=0.3)
plt.show()

print("观察：采样分布近似均匀，时间上不相邻的经验被混合在一起")

## 4. 目标网络

### 4.1 训练不稳定问题

在标准 Q-Learning 中，TD 目标是：

$$y = r + \gamma \max_{a'} Q(s', a'; \theta)$$

问题：$\theta$ 每步都在更新，导致目标 $y$ 不断变化（追逐移动目标）。

### 4.2 目标网络解决方案

使用独立的目标网络 $\theta^-$ 计算 TD 目标：

$$y = r + \gamma \max_{a'} Q(s', a'; \theta^-)$$

$\theta^-$ 周期性地从 $\theta$ 复制（硬更新）或软更新：

$$\theta^- \leftarrow \tau \theta + (1-\tau) \theta^-$$

In [ ]:
# 演示目标网络的稳定性作用

def simulate_training_stability(use_target_network=True, update_period=10):
    """
    模拟训练过程中的目标稳定性
    """
    np.random.seed(42)
    
    # 模拟网络参数
    theta = 1.0
    theta_target = 1.0
    
    # 记录历史
    theta_history = []
    target_history = []
    
    for step in range(100):
        # 模拟梯度更新（带噪声）
        gradient = 0.1 * (np.random.randn() - 0.1 * theta)
        theta += gradient
        
        if use_target_network:
            # 周期性更新目标网络
            if step % update_period == 0:
                theta_target = theta
            target = theta_target
        else:
            target = theta
        
        theta_history.append(theta)
        target_history.append(target)
    
    return theta_history, target_history

# 对比有无目标网络
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 无目标网络
theta_no_target, target_no_target = simulate_training_stability(use_target_network=False)
axes[0].plot(theta_no_target, label='θ (online)', alpha=0.8)
axes[0].plot(target_no_target, label='target', alpha=0.8, linestyle='--')
axes[0].set_title('Without Target Network\n(Target changes every step)')
axes[0].set_xlabel('Training Step')
axes[0].set_ylabel('Parameter Value')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# 有目标网络
theta_with_target, target_with_target = simulate_training_stability(use_target_network=True, update_period=10)
axes[1].plot(theta_with_target, label='θ (online)', alpha=0.8)
axes[1].plot(target_with_target, label='θ⁻ (target)', alpha=0.8, linestyle='--')
axes[1].set_title('With Target Network (update every 10 steps)\n(Target is stable)')
axes[1].set_xlabel('Training Step')
axes[1].set_ylabel('Parameter Value')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("观察：目标网络提供了稳定的训练目标（阶梯状变化）")

## 5. ε-贪婪探索

### 5.1 探索-利用权衡

强化学习面临经典的探索-利用困境：
- **利用 (Exploitation)**: 选择当前已知最优动作
- **探索 (Exploration)**: 尝试新动作以发现更好的策略

### 5.2 ε-贪婪策略

$$a = \begin{cases}
\text{随机动作} & \text{概率 } \epsilon \\
\arg\max_a Q(s, a) & \text{概率 } 1 - \epsilon
\end{cases}$$

通常 $\epsilon$ 随训练衰减：从高探索逐渐转向高利用。

In [ ]:
# 可视化不同的 ε 衰减策略

def linear_decay(step, start=1.0, end=0.01, decay_steps=10000):
    """线性衰减"""
    return max(end, start - (start - end) * step / decay_steps)

def exponential_decay(step, start=1.0, end=0.01, decay_rate=0.9995):
    """指数衰减"""
    return max(end, start * (decay_rate ** step))

def step_decay(step, start=1.0, end=0.01, decay_steps=5000):
    """阶梯衰减"""
    levels = [1.0, 0.5, 0.2, 0.1, 0.01]
    idx = min(step // decay_steps, len(levels) - 1)
    return levels[idx]

# 绘制衰减曲线
steps = np.arange(20000)

plt.figure(figsize=(10, 6))
plt.plot(steps, [linear_decay(s) for s in steps], label='Linear Decay')
plt.plot(steps, [exponential_decay(s) for s in steps], label='Exponential Decay')
plt.plot(steps, [step_decay(s) for s in steps], label='Step Decay')

plt.xlabel('Training Step')
plt.ylabel('ε (Exploration Rate)')
plt.title('ε-Greedy Exploration Schedules')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print("不同衰减策略的特点:")
print("- 线性衰减: 简单直观，衰减速度恒定")
print("- 指数衰减: 初期快速衰减，后期缓慢")
print("- 阶梯衰减: 阶段性调整，便于控制")

## 6. DQN 变体对比

### 6.1 Double DQN

**问题**: 标准 DQN 存在 Q 值过估计问题

标准 DQN 的 max 操作同时用于选择和评估动作：
$$y = r + \gamma \max_{a'} Q(s', a'; \theta^-)$$

**Double DQN 解决方案**:
$$y = r + \gamma Q(s', \arg\max_{a'} Q(s', a'; \theta); \theta^-)$$

- 在线网络选择动作
- 目标网络评估动作

### 6.2 Dueling DQN

将 Q 值分解为状态价值和优势函数：
$$Q(s, a) = V(s) + A(s, a) - \frac{1}{|\mathcal{A}|}\sum_{a'} A(s, a')$$

### 6.3 算法对比表

| 变体 | 核心改进 | 优点 |
|------|---------|------|
| DQN | 经验回放 + 目标网络 | 基础稳定方法 |
| Double DQN | 分离选择与评估 | 缓解过估计 |
| Dueling DQN | V + A 分解 | 更好的状态表示 |
| PER | 优先级采样 | 提高样本效率 |
| Noisy DQN | 参数化探索 | 更好的探索 |

In [ ]:
# 演示 Q 值过估计问题

def demonstrate_overestimation():
    """
    演示 max 操作导致的过估计
    
    假设真实 Q 值为 0，但估计值有噪声
    """
    np.random.seed(42)
    
    num_actions = 10
    num_trials = 1000
    
    # 不同噪声水平
    noise_levels = np.linspace(0.1, 2.0, 20)
    
    # 真实值为 0
    true_q = 0.0
    
    max_estimates = []
    mean_estimates = []
    
    for noise in noise_levels:
        max_vals = []
        mean_vals = []
        
        for _ in range(num_trials):
            # Q 估计 = 真实值 + 噪声
            q_estimates = true_q + noise * np.random.randn(num_actions)
            
            max_vals.append(np.max(q_estimates))
            mean_vals.append(np.mean(q_estimates))
        
        max_estimates.append(np.mean(max_vals))
        mean_estimates.append(np.mean(mean_vals))
    
    return noise_levels, max_estimates, mean_estimates

noise_levels, max_est, mean_est = demonstrate_overestimation()

plt.figure(figsize=(10, 6))
plt.plot(noise_levels, max_est, 'b-o', label='max Q (overestimated)', markersize=4)
plt.plot(noise_levels, mean_est, 'g-s', label='mean Q', markersize=4)
plt.axhline(y=0, color='r', linestyle='--', label='True Q = 0')

plt.xlabel('Estimation Noise (σ)')
plt.ylabel('Estimated Value')
plt.title('Q-Value Overestimation due to max Operation\n(True Q = 0 for all actions)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print("观察：噪声越大，max 操作导致的过估计越严重")
print("Double DQN 通过分离选择和评估来缓解这个问题")

## 7. 实验与分析

### 7.1 在 CartPole 上训练 DQN

In [ ]:
# 创建环境
env_name = 'CartPole-v1'
train_py_env = suite_gym.load(env_name)
eval_py_env = suite_gym.load(env_name)

train_env = tf_py_environment.TFPyEnvironment(train_py_env)
eval_env = tf_py_environment.TFPyEnvironment(eval_py_env)

print(f"环境: {env_name}")
print(f"观测空间: {train_env.observation_spec()}")
print(f"动作空间: {train_env.action_spec()}")

In [ ]:
# 配置超参数
num_iterations = 5000
initial_collect_steps = 1000
collect_steps_per_iteration = 1
replay_buffer_capacity = 100000
batch_size = 64
learning_rate = 1e-3
gamma = 0.99
target_update_period = 200
epsilon_greedy = 0.1
log_interval = 500
eval_interval = 1000
num_eval_episodes = 10

# 创建 Q 网络
fc_layer_params = (100, 50)
q_net = q_network.QNetwork(
    train_env.observation_spec(),
    train_env.action_spec(),
    fc_layer_params=fc_layer_params
)

# 创建优化器
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

# 创建 DQN Agent
train_step_counter = tf.Variable(0, dtype=tf.int64)

agent = dqn_agent.DqnAgent(
    train_env.time_step_spec(),
    train_env.action_spec(),
    q_network=q_net,
    optimizer=optimizer,
    td_errors_loss_fn=common.element_wise_squared_loss,
    train_step_counter=train_step_counter,
    gamma=gamma,
    epsilon_greedy=epsilon_greedy,
    target_update_period=target_update_period
)

agent.initialize()
print("DQN Agent 创建完成")

In [ ]:
# 创建回放缓冲区
replay_buffer = tf_uniform_replay_buffer.TFUniformReplayBuffer(
    data_spec=agent.collect_data_spec,
    batch_size=train_env.batch_size,
    max_length=replay_buffer_capacity
)

# 数据收集函数
def collect_step(environment, policy, buffer):
    time_step = environment.current_time_step()
    action_step = policy.action(time_step)
    next_time_step = environment.step(action_step.action)
    traj = trajectory.from_transition(time_step, action_step, next_time_step)
    buffer.add_batch(traj)

def evaluate_policy(environment, policy, num_episodes=10):
    total_return = 0.0
    for _ in range(num_episodes):
        time_step = environment.reset()
        episode_return = 0.0
        while not time_step.is_last():
            action_step = policy.action(time_step)
            time_step = environment.step(action_step.action)
            episode_return += time_step.reward.numpy()[0]
        total_return += episode_return
    return total_return / num_episodes

# 预填充缓冲区
random_policy = random_tf_policy.RandomTFPolicy(
    train_env.time_step_spec(),
    train_env.action_spec()
)

train_env.reset()
for _ in range(initial_collect_steps):
    collect_step(train_env, random_policy, replay_buffer)

print(f"缓冲区初始大小: {replay_buffer.num_frames().numpy()}")

In [ ]:
# 训练循环
dataset = replay_buffer.as_dataset(
    num_parallel_calls=3,
    sample_batch_size=batch_size,
    num_steps=2
).prefetch(3)

iterator = iter(dataset)
agent.train = common.function(agent.train)

# 记录历史
returns = []
losses = []

# 初始评估
avg_return = evaluate_policy(eval_env, agent.policy, num_eval_episodes)
returns.append(avg_return)
print(f"Step 0: Avg Return = {avg_return:.2f}")

# 训练
train_env.reset()

for iteration in range(1, num_iterations + 1):
    # 收集数据
    for _ in range(collect_steps_per_iteration):
        collect_step(train_env, agent.collect_policy, replay_buffer)
    
    # 训练
    experience, _ = next(iterator)
    train_loss = agent.train(experience).loss
    losses.append(train_loss.numpy())
    
    step = agent.train_step_counter.numpy()
    
    if step % log_interval == 0:
        print(f"Step {step}: Loss = {train_loss:.4f}")
    
    if step % eval_interval == 0:
        avg_return = evaluate_policy(eval_env, agent.policy, num_eval_episodes)
        returns.append(avg_return)
        print(f"Step {step}: Avg Return = {avg_return:.2f}")

print("\n训练完成!")

In [ ]:
# 可视化训练结果
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 回报曲线
ax1 = axes[0]
eval_steps = [0] + list(range(eval_interval, num_iterations + 1, eval_interval))
ax1.plot(eval_steps, returns, 'b-o', linewidth=2, markersize=4)
ax1.set_xlabel('Training Step')
ax1.set_ylabel('Average Return')
ax1.set_title('DQN Training: Episode Returns')
ax1.grid(True, alpha=0.3)
ax1.axhline(y=500, color='g', linestyle='--', label='Max Score')
ax1.legend()

# 损失曲线
ax2 = axes[1]
window = 50
smoothed = np.convolve(losses, np.ones(window)/window, mode='valid')
ax2.plot(smoothed, 'r-', linewidth=1, alpha=0.8)
ax2.set_xlabel('Training Step')
ax2.set_ylabel('Loss (smoothed)')
ax2.set_title('DQN Training: TD Loss')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 总结

### DQN 关键要点

1. **Q 函数逼近**: 使用神经网络替代 Q 表，处理高维状态

2. **经验回放**:
   - 打破样本相关性
   - 提高数据利用效率
   - 使训练更稳定

3. **目标网络**:
   - 提供稳定的训练目标
   - 防止追逐移动目标
   - 周期性或软更新

4. **ε-贪婪探索**:
   - 平衡探索与利用
   - 衰减策略很重要

### 数学公式回顾

**贝尔曼方程**:
$$Q^*(s, a) = \mathbb{E}[r + \gamma \max_{a'} Q^*(s', a')]$$

**DQN 损失函数**:
$$L(\theta) = \mathbb{E}[(r + \gamma \max_{a'} Q_{\theta^-}(s', a') - Q_\theta(s, a))^2]$$

### 下一步

- 学习 Actor-Critic 方法（SAC、PPO）
- 探索 DQN 变体（Double DQN、Dueling DQN）
- 应用到更复杂的环境